In [38]:
import json
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [2]:
# Set your Spotify app credentials
SPOTIPY_CLIENT_ID = '330a4427250d4e1aab13cb14859ead7b'
SPOTIPY_CLIENT_SECRET = '07bb4e0d8f12486482224ed60bd7da7a'
SPOTIPY_REDIRECT_URI = 'http://localhost:3000'  # e.g., 'http://localhost:8080'

In [43]:
# Authenticate with Spotify using OAuth
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                               client_secret=SPOTIPY_CLIENT_SECRET,
                                               redirect_uri=SPOTIPY_REDIRECT_URI,
                                               scope="user-library-read,user-top-read"))

In [23]:
offset = 0

tracks = []
while True:
    results = sp.current_user_saved_tracks(offset=offset, limit=50)
    
    if not results['items']:
        break

    for track in results['items']:
        tracks.append(track)
    
    offset = offset + 50

In [21]:
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

In [31]:
for idx, track in enumerate(tracks):
    if idx % 10 == 0:
        print(idx, track['track']['artists'][0]['name'], ' - ', track['track']['name'], end="\n")
    else:
        print(idx, track['track']['artists'][0]['name'], ' - ', track['track']['name'], end=" ")
        

0 AJR  -  Burn the House Down
1 Marshmello  -  One Thing Right 2 Eminem  -  Beautiful 3 Eminem  -  Rap God 4 Macklemore  -  Glorious (feat. Skylar Grey) 5 Trevor Daniel  -  Falling 6 NF  -  PANDEMONIUM 7 NF  -  HAPPY 8 NF  -  Time 9 Eminem  -  When I'm Gone 10 Eminem  -  River (feat. Ed Sheeran)
11 *NSYNC  -  Bye Bye Bye - From Deadpool and Wolverine Soundtrack 12 Pitbull  -  Feel This Moment (feat. Christina Aguilera) 13 Maroon 5  -  This Love 14 Eagles  -  Hotel California - 2013 Remaster 15 Aerosmith  -  Dream On 16 KISS  -  I Was Made For Lovin' You 17 Hatikva 6  -  גיבורי על 18 Depeche Mode  -  Strangelove 19 Twenty One Pilots  -  Leave the City 20 Pitbull  -  Time of Our Lives
21 Panic! At The Disco  -  House of Memories 22 The Beatles  -  All My Loving - Remastered 2009 23 Kygo  -  Firestone 24 Alok  -  Deep Down (feat. Never Dull) 25 Martin Solveig  -  Do It Right 26 Scorpions  -  Wind Of Change 27 Lost Frequencies  -  Black Friday (pretty like the sun) 28 Mark Ronson  -  Uptow

In [32]:
tracks[0]

{'added_at': '2024-09-26T13:38:15Z',
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6s22t5Y3prQHyaHWUN1R1C'},
     'href': 'https://api.spotify.com/v1/artists/6s22t5Y3prQHyaHWUN1R1C',
     'id': '6s22t5Y3prQHyaHWUN1R1C',
     'name': 'AJR',
     'type': 'artist',
     'uri': 'spotify:artist:6s22t5Y3prQHyaHWUN1R1C'}],
   'available_markets': ['AR',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'NL',
    'NI',
    'NO',
    'PA',
    'PY',
    'PE',
    'PH',
    'PL',
    'PT',
    'SG',
    'SK',
    'ES',
    'SE',
    'CH',
    'TW',
    'TR',
    'UY',
    'GB',
    'AD',
    'LI',
    'MC',
    'ID',
    'JP',
    'TH',
    'VN',
    'RO',
    '

In [41]:
print(json.dumps(tracks[0]['track']))

{"album": {"album_type": "album", "artists": [{"external_urls": {"spotify": "https://open.spotify.com/artist/6s22t5Y3prQHyaHWUN1R1C"}, "href": "https://api.spotify.com/v1/artists/6s22t5Y3prQHyaHWUN1R1C", "id": "6s22t5Y3prQHyaHWUN1R1C", "name": "AJR", "type": "artist", "uri": "spotify:artist:6s22t5Y3prQHyaHWUN1R1C"}], "available_markets": ["AR", "AT", "BE", "BO", "BR", "BG", "CL", "CO", "CR", "CY", "CZ", "DK", "DO", "DE", "EC", "EE", "SV", "FI", "FR", "GR", "GT", "HN", "HK", "HU", "IS", "IE", "IT", "LV", "LT", "LU", "MY", "MT", "NL", "NI", "NO", "PA", "PY", "PE", "PH", "PL", "PT", "SG", "SK", "ES", "SE", "CH", "TW", "TR", "UY", "GB", "AD", "LI", "MC", "ID", "JP", "TH", "VN", "RO", "IL", "ZA", "SA", "AE", "BH", "QA", "OM", "KW", "EG", "MA", "DZ", "TN", "LB", "JO", "PS", "IN", "BY", "KZ", "MD", "UA", "AL", "BA", "HR", "ME", "MK", "RS", "SI", "KR", "BD", "PK", "LK", "GH", "KE", "NG", "TZ", "UG", "AG", "AM", "BS", "BB", "BZ", "BT", "BW", "BF", "CV", "CW", "DM", "GM", "GE", "GD", "GW", "GY",

In [49]:
top_tracks = sp.current_user_top_tracks(time_range='short_term')

In [50]:
for item in top_tracks['items']:
    print(json.dumps(item))

{"album": {"album_type": "ALBUM", "artists": [{"external_urls": {"spotify": "https://open.spotify.com/artist/6fOMl44jA4Sp5b9PpYCkzz"}, "href": "https://api.spotify.com/v1/artists/6fOMl44jA4Sp5b9PpYCkzz", "id": "6fOMl44jA4Sp5b9PpYCkzz", "name": "NF", "type": "artist", "uri": "spotify:artist:6fOMl44jA4Sp5b9PpYCkzz"}], "available_markets": ["AR", "AU", "AT", "BE", "BO", "BR", "BG", "CA", "CL", "CO", "CR", "CY", "CZ", "DK", "DO", "DE", "EC", "EE", "SV", "FI", "FR", "GR", "GT", "HN", "HK", "HU", "IS", "IE", "IT", "LV", "LT", "LU", "MY", "MT", "MX", "NL", "NZ", "NI", "NO", "PA", "PY", "PE", "PH", "PL", "PT", "SG", "SK", "ES", "SE", "CH", "TW", "TR", "UY", "US", "GB", "AD", "LI", "MC", "ID", "JP", "TH", "VN", "RO", "IL", "ZA", "SA", "AE", "BH", "QA", "OM", "KW", "EG", "MA", "DZ", "TN", "LB", "JO", "PS", "IN", "KZ", "MD", "UA", "AL", "BA", "HR", "ME", "MK", "RS", "SI", "KR", "BD", "PK", "LK", "GH", "KE", "NG", "TZ", "UG", "AG", "AM", "BS", "BB", "BZ", "BT", "BW", "BF", "CV", "CW", "DM", "FJ", 